In [1]:
import itertools
import nltk
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
%load_ext autoreload
%autoreload 2

In [9]:
# Read
print "Read txt file..."

with open("data/ppd/test.txt","rb") as f:
    reader = f.readlines()
#     sentences = itertools.chain(*nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader)
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in reader]

Read txt file...


In [7]:
print "Parsed %d sentences." % len(sentences)

Parsed 57 sentences.


In [10]:
print sentences[0]
print sentences[1]

SENTENCE_START Merus sank into his chair with relief. 
 SENTENCE_END
SENTENCE_START He listened for the satisfying crackling sound of sinking into the chair's soft material. 
 SENTENCE_END


###Tokenize the sentences into words

In [11]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [12]:
tokenized_sentences[0]

['SENTENCE_START',
 'Merus',
 'sank',
 'into',
 'his',
 'chair',
 'with',
 'relief',
 '.',
 'SENTENCE_END']

###Count the word freq

In [13]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

### Get the most common words and build index_to_word & word_to_index

In [16]:
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [32]:
print type(vocab)
print len(vocab)

<type 'list'>
725


In [20]:
for i in xrange(10):
    print i,index_to_word[i]

0 the
1 .
2 ,
3 SENTENCE_START
4 SENTENCE_END
5 of
6 and
7 a
8 his
9 in


In [21]:
print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

Using vocabulary size 8000.
The least frequent word in our vocabulary is 'original' and appeared 1 times.


###Replace all words not in dict with unknown

In [22]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [23]:
print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]


Example sentence: 'SENTENCE_START Merus sank into his chair with relief. 
 SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'Merus', 'sank', 'into', 'his', 'chair', 'with', 'relief', '.', 'SENTENCE_END']'


### Create the training data

###Convert the input to an array
np.asarray

In [92]:
import numpy as np
X_train = np.asarray([ np.asarray([word_to_index[w] for w in sent[:-1]]) for sent in tokenized_sentences])
y_train = np.asarray([ np.asarray([word_to_index[w] for w in sent[1:]]) for sent in tokenized_sentences])

In [93]:
print X_train.shape
print y_train.shape

(57L,)
(57L,)


In [94]:
print word_to_index["The"]
print word_to_index["reviews"]

13
115


In [95]:
print sentences[10]
print X_train[10]

SENTENCE_START The reviews on his profile were equally stellar:"Merus is fantastic. 
 SENTENCE_END
[  3  13 115  27   8 308  16 657 646  96  61  18 560 260   1]


####Start to train

In [96]:
from rnnlm import RNNLM
hdim = 100
np.random.seed(10)
L0 = np.zeros((725,hdim))
model = RNNLM(L0,U0=L0,alpha=0.01,rseed=10,bptt=4)
# model.grad_check(np.array([1,2,3]),np.array([2,3,4]))

In [98]:
from rnnlm import RNNLM
epoch = 1
nn = len(y_train)
X = X_train[:nn]
Y = y_train[:nn]
idxiter = np.random.permutation(range(nn)*epoch)
print "Using %d train example" % nn

model.train_sgd(X,Y,idxiter=idxiter)

Using 57 train example
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 5.70736
  [57]: mean loss 5.65913
SGD complete: 57 examples in 1.83 seconds.


[(0, 5.7073626347925526), (57, 5.6591337690085517)]

In [101]:
def seq_to_words(seq):
    return [index_to_word[s] for s in seq]
    
seq, J = model.generate_sequence(word_to_index[sentence_start_token], 
                                 word_to_index[sentence_end_token], 
                                 maxlen=30)
print J
# print seq
print " ".join(seq_to_words(seq))

129.595818403
SENTENCE_START theoretical of in a process into a his to , spotted sunk He was . up agent stretch eyes lucrative PhD Task light SENTENCE_END
